# Initialize things 

In [16]:
#import things

import cv2
import numpy  as np
from collections import deque
import time


In [17]:
from keras.models import load_model
cnn_model = load_model('emnist_cnn_model.h5')

In [18]:
prediction_dict = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l',
           12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w',
           23: 'x', 24: 'y', 25: 'z', 26: ''}

In [19]:
# define color limits

red_upper = (170,100,60)
red_lower = (180,255,255)

points = deque(maxlen = 512)
kernel = np.ones((5, 5), np.uint8)

#define blackboard and alphabets
blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
alphabet = np.zeros((300, 300, 3), dtype=np.uint8)

prediction = 26 #default prediction
center = None


# Capture the written letter

In [20]:

cap = cv2.VideoCapture(0)
    
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    #hsv
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #red
    red = cv2.inRange(hsv, red_upper, red_lower)
    #Preprocessing the  image
    red = cv2.morphologyEx(red, cv2.MORPH_OPEN, kernel) 
    red_median = cv2.medianBlur(red, 15)
    
   
    #find contours
    contours, _ = cv2.findContours(red_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        contour_list = sorted(contours, key = cv2.contourArea, reverse = True)
        contour = contour_list[0]
        #find centroid
        M = cv2.moments(contour) #M will be a dict
        try:
            centroid_x = int(M["m10"]/M["m00"])
            centroid_y = int(M["m01"]/M["m00"])
        except:
            pass
        
        centroid = (centroid_x, centroid_y)
        #append centroid in points
        points.appendleft(centroid)
        #draw circle
        ((x,y), r) = cv2.minEnclosingCircle(contour)
        cv2.circle(frame, (int(x),int(y)), int(r), (255,215,0), 2)
   
    elif (len(contours)==0):
        if(len(points) != 0): #means there is some letter which was written and now the object is taken away
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur = cv2.medianBlur(blackboard_gray, 15)
            blur = cv2.GaussianBlur(blur, (5, 5), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            
            blackboard_cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
            if len(blackboard_cnts) >= 1:
                #sorting all the contours to get the largest one
                contour = sorted(blackboard_cnts, key = cv2.contourArea, reverse = True)[0]

                if cv2.contourArea(contour) > 1000: #applying area condition to prevent noise from getting selected
                    x, y, w, h = cv2.boundingRect(contour)
                    alphabet = blackboard_gray[y-10:y + h + 10, x-10:x + w + 10]
                
                    try:
                        final_alphabet = cv2.resize(alphabet, (28,28))
                        # this is the final alphabet image 
                    except cv2.error as e:
                        points = deque(maxlen=512)
                        blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
                        continue
                        
                    final_alphabet = np.array(final_alphabet)
                    final_alphabet = final_alphabet.astype("float32")/255
                    
                    prediction = cnn_model.predict(final_alphabet.reshape(1,28,28,1))[0]
                    prediction = np.argmax(prediction)
                    print(f"The predicted alphabet was {prediction_dict[int(prediction)]}")#final output from model
                
        # Empty the points deque and the blackboard
        
        blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
        points = deque(maxlen=512)
        
    #connecting the points with the lines
    for i in range(1, len(points)):
        if points[i - 1] is None or points[i] is None:
            continue
        cv2.line(frame, tuple(map(int,points[i-1])),tuple(map(int,points[i])) ,(0,0,0), 2)
        cv2.line(blackboard, tuple(map(int,points[i-1])), tuple(map(int,points[i])), (255,255,255), 8)
        
    # Show the frame
    

    cv2.putText(frame, "Convolution Neural Network:  " + str(prediction_dict[int(prediction)]), (10, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.imshow("alphabets Recognition Real Time", frame) 
    
    if cv2.waitKey(1) &  0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()


The predicted alphabet was c


# Theme(Movies/Songs) selection followed by Movie/Genre Selection

In [ ]:
import pyttsx3
import pandas as pd
import random
engine = pyttsx3.init()


image = cv2.imread('start.jpg')

#adding a function to get the coordinates of double click
def get_coordinates(event,x,y,flags,param):
    global mouseX,mouseY
    if event == cv2.EVENT_LBUTTONDBLCLK:
        cv2.circle(image,(x,y),10,(255,0,0),-1)
        mouseX,mouseY = x,y
cv2.namedWindow('Start')
cv2.setMouseCallback('Start',get_coordinates)

#selecting theme to go ahead with
def select_theme(x,y):
    try:
        if(150<y<200):
            #run the code for the movie theme
            
            titles = pd.read_csv('titles.csv')#file containing movie names and years

            #extracting content from the file
            initial = prediction_dict[int(prediction)]
            list_of_initial = titles[titles["title"].str.startswith(initial.upper())]
            samples = list_of_initial.sample(n = 3).sort_values('year',ascending = False)
            
            names = [samples["title"][int(index)] for index in samples.index]
            years = [samples["year"][int(index)]for index in samples.index]
            content = [names[i] +"   "+ str(years[i])  for i in range(len(names))]

            #randomizing image to be shown
            i  =random.randrange(1,6)
            img_name = f"image_{i}.jpg"
            image = cv2.imread(img_name,cv2.IMREAD_UNCHANGED)

            # perform the resizing and add some contours to it
            r = 555.0 / image.shape[0]
            dim = (int(image.shape[1] * r), 555)
            image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
            image= cv2.drawMarker(image,(140,75),(44,15,66),20,3,8)
            image[100:300,130:135] = [145, 11, 138]

             #adding a function to get the coordinates of double click
            def get_coordinates(event,x,y,flags,param):
                global mouseX,mouseY
                if event == cv2.EVENT_LBUTTONDBLCLK:
                    cv2.circle(image,(x,y),10,(255,0,0),-1)
                    mouseX,mouseY = x,y
            cv2.namedWindow('Recommendations')
            cv2.setMouseCallback('Recommendations',get_coordinates)

            #select movie based on coordinates of double click
            def select_movie(x,y):
                try:
                    assert (140<x<700)
                    selected = 0 #y default the first movie will be selected

                    #selecting index based on y coordinate of click
                    if(160<y<200):
                        selected =1
                    elif(210<y<240):
                        selected=2
                    search_movie(selected)

                except:
                    pass


            #search on google
            def search_movie(selected):
                    import webbrowser
                    #initializing movie to be searched on google
                    movie = names[selected]
                    url = f"https://www.google.com/search?q={movie}&sxsrf=ALeKk00QzMIt-ygt6jWiqamXPuhV7tYvZw%3A1626203284684&source=hp&ei=lOTtYJ7cJ4OC4-EPkdeziAI&iflsig=AINFCbYAAAAAYO3ypN-BWGvrEtBzNihkTPIQTesfZlaF&oq=wakand&gs_lcp=Cgdnd3Mtd2l6EAMyBwgAELEDEEMyBwguELEDEEMyBAgAEEMyBQgAELEDMgIIADIFCAAQsQMyAggAMgIIADICCAAyAggAOgcIIxDqAhAnOgQIIxAnOgUIABCRAjoICAAQsQMQkQI6CAgAELEDEIMBOgcIABDJAxBDOgcIABCHAhAUOgUILhCxAzoICC4QxwEQrwE6AgguOgUILhCTAjoECAAQCjoECC4QQ1C1KliwOmCrPGgCcAB4AYABtAmIAZUikgEJMC4zLjYtMy4xmAEAoAEBqgEHZ3dzLXdperABCg&sclient=gws-wiz&ved=0ahUKEwie9vSo3-DxAhUDwTgGHZHrDCEQ4dUDCAc&uact=5"
                    try :
                        webbrowser.open(url) 
                    except:
                        pass


            while True:
                cv2.putText(image, "WE RECOMMEND YOU:", (150,80), cv2.FONT_HERSHEY_COMPLEX, 0.8, (84,14,150), 2)
                for i in range(len(names)):
                    cv2.putText(image, content[i], (150,130+50*i), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (112,12,71), 2)
                cv2.imshow("Recommendations", image)
                if cv2.waitKey(1) & 0xFF == ord("s"):
                    engine.setProperty('rate', 175)     # setting up new voice rate
                    volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
                    engine.setProperty('volume',2) 
                    engine.say("\n".join(names))
                    engine.runAndWait()

                    break
                elif cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                elif cv2.waitKey(1) & 0xFF == ord("a"):       
                    select_movie(mouseX,mouseY)


            cv2.destroyWindow("Recommendations")

        if(240<y<300):
            
            #run the code for the song theme           
            
            songs = pd.read_csv('songs.csv')#file containing songs
            #extracting content from the file
            initial = prediction_dict[int(prediction)]
            song_list =songs[songs.SPOTIFY_URL!="nan"]
            song_list =songs[songs.TITLE.str.startswith(initial.upper())]
            song_list=song_list.sample(n=1)
            name = [song_list["TITLE"][int(index)] for index in song_list.index]
            year = [song_list["YEAR"][int(index)]for index in song_list.index]
            theme = [song_list["THEME"][int(index)]for index in song_list.index]
            artist =[song_list["ARTIST"][int(index)]for index in song_list.index]
            song_link =[song_list["SPOTIFY_URL"][int(index)]for index in song_list.index]

            content = [theme[0],f"Song : {name[0]}","Year "+ str( year[0]),f"Artist : {artist[0]}"]
            
            #randomizing image to be shown
            genre = theme[0]
            img_name = f"{genre}.jpg"
            image = cv2.imread(img_name,cv2.IMREAD_UNCHANGED)

            # perform the resizing and add some contours to it
            r = 555.0 / image.shape[0]
            dim = (int(image.shape[1] * r), 555)
            image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
            image= cv2.drawMarker(image,(140,75),(44,15,66),20,3,8)
            image[150:300,130:135] = [145, 11, 138]
            
            #adding a function to get the coordinates of double click
            def get_coordinates(event,x,y,flags,param):
                global mouseX,mouseY
                if event == cv2.EVENT_LBUTTONDBLCLK:
                    cv2.circle(image,(x,y),10,(255,0,0),-1)
                    mouseX,mouseY = x,y
            cv2.namedWindow('SongRecommendations')
            cv2.setMouseCallback('SongRecommendations',get_coordinates)
            #select movie based on coordinates of double click
            def open_spotify(x,y):
                try:
                    assert (140<x<700)
                    selected = False

                    #checking if user wants to play song based on coordinates of click
                    if(150<y<210):
                        selected=True
                    play_song(selected)

                except:
                    pass


            #play song on spotify
            def play_song(selected):
                    import webbrowser
                    #initializing movie to be searched on google
                    if(selected):
                        url = f"{song_link[0]}"
                        try :
                            webbrowser.open(url) 
                        except:
                            pass


            while True:
                cv2.putText(image, f"GENRE: {content[0]}", (200,80), cv2.FONT_HERSHEY_DUPLEX, 0.8, (243, 129, 109), 2)

                cv2.putText(image, "YOU SHOULD LISTEN TO :", (150,130), cv2.FONT_HERSHEY_COMPLEX, 0.8, (240, 217, 217), 2)

                for i in range(1,len(content)):
                    cv2.putText(image, content[i], (150,150+50*i), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (216, 89, 238), 2)
                cv2.imshow("SongRecommendations", image)
                if cv2.waitKey(1) & 0xFF == ord("s"):
                    engine.setProperty('rate', 175)     # setting up new voice rate
                    volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
                    engine.setProperty('volume',2) 
                    engine.say(name[0])
                    engine.runAndWait()

                elif cv2.waitKey(1) & 0xFF == ord("q"):
                    break
                elif cv2.waitKey(1) & 0xFF == ord("a"):       
                    open_spotify(mouseX, mouseY)
                    
            cv2.destroyWindow("SongRecommendations")


    except:
        pass
       
                
while True:

    cv2.imshow("Start",image)

    if cv2.waitKey(1) & 0xFF == ord("a"):
        select_theme(mouseX,mouseY)
        
        break    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        
        break    
        

cv2.destroyWindow("Start")